In [2]:
import numpy as np
import matplotlib.pyplot as plt

import discretize

from SimPEG import seismic

from SimPEG import (
    maps,
    utils,
    survey,
    data,
    data_misfit,
    directives,
    optimization,
    regularization,
    inverse_problem,
    inversion,
)

/var/folders/r2/7lcfqppd1sg_grmh2b54b79r0000gn/T/ipykernel_54024/3802917825.py:6: FutureWarning: Importing `SimPEG` is deprecated. please import from `simpeg`.
  from SimPEG import seismic


In [3]:
extent_x = 200 
extent_y = 400

dx = 10
dy = 10

nx = int(np.ceil(extent_x / dx))
ny = int(np.ceil(extent_y / dy))

hx = np.ones(nx) * dx
hy = np.ones(ny) * dy

# n_src = 10
# y_src = np.linspace(dy/2, extent_y-dy/2, n_src)

n_src = 16
n_rec = 16

mesh = discretize.TensorMesh([hx, hy])
mesh

TensorMesh: 800 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x     20          0.00        200.00     10.00     10.00    1.00
   y     40          0.00        400.00     10.00     10.00    1.00

In [4]:
# source_locations = np.c_[np.zeros_like(y_src), y_src]
# receiver_locations = np.c_[np.zeros_like(y_src) + mesh.nodes_x[-1], y_src]

In [5]:
x_src = np.linspace(dx, extent_x - dx, int(np.sqrt(n_src)))
y_src = np.linspace(dy, extent_y - dy, int(np.sqrt(n_src)))
xv_src, yv_src = np.meshgrid(x_src, y_src)
source_locations = np.c_[xv_src.ravel(), yv_src.ravel()] 

x_rec = np.linspace(dx, extent_x - dx, int(np.sqrt(n_rec)))
y_rec = np.linspace(dy, extent_y - dy, int(np.sqrt(n_rec)))
xv_rec, yv_rec = np.meshgrid(x_rec, y_rec)
receiver_locations = np.c_[xv_rec.ravel(), yv_rec.ravel()] 

In [6]:
rx = survey.BaseRx(receiver_locations)

# source_list = [
#     survey.BaseSrc(
#         location=source_locations[i, :], receiver_list=[rx]
#     )
#     for i in range(y_src.size)
# ]

source_list = []

# for i in range(y_src.size):
#     source_list.append(
#         survey.BaseSrc(
#             location=source_locations[i, :], receiver_list=[rx]
#         )
#     )

for i in range(source_locations.shape[0]):
    source_list.append(
        survey.BaseSrc(
            location=source_locations[i, :],
            receiver_list=[rx]
        )
    )

In [17]:
tomo_survey = seismic.straight_ray_tomography.Survey(source_list)

In [23]:
simulation = seismic.straight_ray_tomography.Simulation(
    mesh=mesh,
    survey=tomo_survey,
    slownessMap=maps.IdentityMap(mesh),
)